In [1]:
import pandas as pd
import numpy as np
import json
import os
import requests
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# Grab the data
path = "C:/Users/Erik/Google Drive (erik.sorensen20@houghton.edu)/DotaAnalysis/versions/7.07d/"

picks_bans = []
radiant_wins = []
for filename in os.listdir(path):
    with open(path + filename,'r') as outfile:
        data = json.load(outfile)
        picks_bans.append(data['picks_bans'])
        radiant_wins.append(data['radiant_win'])

In [3]:
#Drop the team and ord data from picks_ban
draft = list(picks_bans)
for i in range(len(draft)):
    try:
        for x in range(len(draft[i])):
            del(draft[i][x]['ord'])
            #del(draft[i][x]['team'])
    except:
        continue

In [4]:
# Combine picks_bans and radiant wins
comb = []
for i in range(len(picks_bans)):
    comb.append([picks_bans[i],radiant_wins[i]])

In [5]:
#Take out the None's in the data
#picks_bans = [x for x in picks_bans if x != None]
#radiant_wins = [x for x in radiant_wins if x != None]
comb = [x for x in comb if x[0] != None]

In [6]:
#with open('C:/Users/Erik/Google Drive (erik.sorensen20@houghton.edu)/DotaAnalysis/analysis/PickBans+Win_707d.json','w') as outfile:
#			json.dump(comb, outfile)

In [7]:
firstTeam = []
[firstTeam.append(x[0][0]['team']) for x in comb]
firstTeam = np.array(firstTeam)
print("Percentage of times team 1 (dire?) got first pick: {:.4f}".format(np.mean(firstTeam)))

Percentage of times team 1 (dire?) got first pick: 0.5974


In [8]:
comb[0][0][2]

{'hero_id': 60,
 'is_pick': False,
 'match_id': 3626698786,
 'order': 2,
 'team': 1}

In [9]:
# We want to convert the picks_bans json file into a 2D data frame where Match_ID is the rows and pick_num is the columns
# To do that we need to encode the info in each pick into one string
#Eg: Ban/Pick-Hero_Num-Team
#Team may not be important or may throw off the association rules (although future analysis it might be interesting analyzing team drafting trends)
df = pd.DataFrame(data={'p1':['B19D'],
                        'p2':['B107R'],
                        'p3':['B60D']})

In [10]:
df

,p1,p2,p3
0,B19D,B107R,B60D


In [11]:
	# Grab current hero data from opendota.api
	page = requests.get('https://api.opendota.com/api/heroes')
	content = page.content

	# Load the json file
	heros = json.loads(content)

In [12]:
comb_with_heros = list(comb)

In [13]:
for i in comb_with_heros:
    for x in i[0]:
        #print(x['hero_id'])
        #For some reason the hero id 24 is skipped so you need to search for the correct id not just index the list of hero names
        x['hero_name'] = [hero['localized_name'] for hero in heros if hero['id'] == x['hero_id']]

In [14]:
comb_with_heros[1][0][5]['hero_name']

['Bounty Hunter']

In [15]:
with open('C:/Users/Erik/Google Drive (erik.sorensen20@houghton.edu)/DotaAnalysis/analysis/DraftsWithHeros_707d.json','w') as outfile:
            json.dump(comb_with_heros, outfile)